In [1]:
import json
import pandas as pd
import MeCab

In [2]:
stories_json = []
with open('./output_stories_0123.json', 'r') as f:
    stories_json = stories_json + json.loads(f.read())

print(len(stories_json))
print(stories_json[1].keys())

839
dict_keys(['index', 'stories_index', 'title', 'broadcasting_date', 'abstracts_list'])


In [3]:
stories_df = pd.DataFrame(stories_json)
stories_df["abstract_joined"] = stories_df["abstracts_list"].str.join(" ")
stories_df.head()

,index,stories_index,title,broadcasting_date,abstracts_list,abstract_joined
0,0325,0,「映画ドラえもん のび太の新魔界大冒険～７人の魔法使い～」,2013年3月15日,[ 何をやってもうまくいかないのび太は、「魔法（まほう）が使えたら便利で楽しくなるにちがいな...,何をやってもうまくいかないのび太は、「魔法（まほう）が使えたら便利で楽しくなるにちがいない...
1,0326,0,「ばくはつコショウ」,2013年4月12日,[ ある朝、ねぼうしたのび太に泣きつかれたドラえもんは、『ばくはつコショウ』を取り出す。この...,ある朝、ねぼうしたのび太に泣きつかれたドラえもんは、『ばくはつコショウ』を取り出す。このコ...
2,0326,1,「何が何でもお花見を」,2013年4月12日,[ みんなから家族でお花見に行った話を聞き、うらやましくてしかたのないのび太。さっそくママに...,みんなから家族でお花見に行った話を聞き、うらやましくてしかたのないのび太。さっそくママにた...
3,0327,0,「やりクリしてハワイ旅行」,2013年4月26日,[ スネ夫からハワイ旅行をじまんされたのび太は、自分もハワイに行きたいとママにたのもうとする...,スネ夫からハワイ旅行をじまんされたのび太は、自分もハワイに行きたいとママにたのもうとするが...
4,0327,1,「しずかちゃんのはごろも」,2013年4月26日,[ 学校で「天女と羽衣（はごろも）」の劇（げき）をやることになったのび太としずか。劇の最後、...,学校で「天女と羽衣（はごろも）」の劇（げき）をやることになったのび太としずか。劇の最後、天...


In [4]:
t = MeCab.Tagger('-d /opt/homebrew/lib/mecab/dic/mecab-ipadic-neologd')
def mecab_tokenizer(text):
    parsed_lines = t.parse(text).split("\n")[:-2]
    surfaces = [l.split('\t')[0] for l in parsed_lines]
    features = [l.split('\t')[1] for l in parsed_lines]
    # 原型を取得
    bases = [f.split(',')[6] for f in features]
    # 品詞を取得
    pos = [f.split(',')[0] for f in features]

    # 各単語を原型に変換する
    token_list = [b if b != '*' else s for s, b in zip(surfaces, bases)]
    # 品詞を絞り込み
    target_pos = ["名詞"]
    token_list = [t for t, p in zip(token_list, pos) if p in target_pos]
    # アルファベットを小文字に統一
    token_list = [t.lower() for t in token_list]

    return [ text for text in filter(lambda x: not x.isascii(), token_list)]

In [5]:
# ストーリーが取得できたものに絞る
stories_df = stories_df[stories_df['abstract_joined'].str.len() > 0] 
sentences = stories_df["abstract_joined"].apply(mecab_tokenizer)
print(sentences)

0      [何, のび太, 魔法, 魔法, 便利, ちがい, もしもボックス, 現実, 世界, 魔法,...
1      [朝, 寝坊, のび太, ドラえもん, 爆発, コショウ, コショウ, 行き先, そば, 人...
2      [みんな, 家族, お花見, 話, しかた, のび太, ママ, パパ, ゴルフ, 予定, の...
3      [スネ夫, ハワイ旅行, じまん, のび太, 自分, ハワイ, ママ, 給料日, 前, お金...
4      [学校, 天女, 羽衣, 羽子, ろ, 劇, げき, こと, のび太, しずか, 劇, 最後...
                             ...                        
834    [星がきれい, 夜, のび太, スネ夫, 家, 前, 庭, 星空, パーティー, 女の子, ...
835    [おなか, 食事, のび太, ご飯, ありがたみ, ママ, のび太, まじめ, 米, パン,...
836    [子供達, 鬼ごっこ, の, のび太, 自分たち, 野郎, みんな, ジャイアン, スネ夫,...
837    [５, 回, 連続, ０, 点, のび太, 来週, テスト, ０, 点, ママ, 鬼, 顔,...
838    [冬, スポーツ, ウインタースポーツ, ランド, のび太, たち, スキー, スケート, ...
Name: abstract_joined, Length: 839, dtype: object


In [16]:
# print(stories_df[stories_df["abstract_joined"].str.contains("殺")]["abstract_joined"])

276    \r\n　町内対抗ドッジボール大会を明日にひかえ、空き地で作戦を練るのび太たち。ところが、秘...
333    親戚ののび四郎おじさんが、久しぶりに遊びにやってくる。動物写真家のおじさんは、いつも外国を飛...
Name: abstract_joined, dtype: object


In [6]:
keywords_dict = {}
for keyword_list in sentences.tolist():
    for keyword in keyword_list:
        if (keyword not in keywords_dict): keywords_dict[keyword] = 1
        else: keywords_dict[keyword] = keywords_dict[keyword] + 1

keywords_df = pd.DataFrame([{
    "key": key,
    "val": val
} for key,val in keywords_dict.items()])
keywords_df = keywords_df.sort_values("val", ascending=False)


In [7]:
pd.set_option('display.max_rows', 500)
keywords_df

,key,val
1,のび太,3916
34,ドラえもん,2419
55,こと,1331
14,の,1241
42,ジャイアン,997
...,...,...
4697,ノビタランド,1
4699,和菓子屋,1
3169,舟,1
3626,春,1


In [8]:
import json
keywords_df.to_csv('doraemon-word-frequencies.csv')